## Práctica 5: Detección de Características

### Ejercicio 1: Desarrolle una aplicación que permita:  

a) A través de la interfaz modificar los parámetros del detector de características SIFT.  
b) Seleccionar un área de interés en una imagen de elección una imagen de naturaleza médica y
una imagen telemétrica.  
c) Buscar esa área de interés (recuádrela en rojo) dentro de diferentes versiones de la imagen de
partida (con cambios de traslación, escala y rotación) [NOTA: Estos cambios se pueden acometer
con un editor de imágenes o con el trabajo hecho en prácticas previas]. Altere mediante la interfaz
las configuraciones de parámetros para mejorar la detección.  
d) Pruebe a hacer lo mismo que en el apartado c) con diferentes grados de deformación de la
imagen. [NOTA: Estos cambios se pueden acometer con un editor de imágenes o con el trabajo
hecho en prácticas previas].  

In [12]:
from tkinter import filedialog
import cv2
from Transformaciones import Transformaciones
import numpy as np

# Función vacía usada como callback por los trackbars (no necesita hacer nada)
def nothing(x):
    pass

def select_roi(img):
    """
    Permite al usuario seleccionar manualmente un área de interés (ROI)
    sobre la imagen original. La ROI será usada posteriormente para
    buscarla en versiones transformadas de la imagen.
    """
    global roi_cords, roi_img
    
    print("Selecionar el Area de Interes (Pulse ENTER para continuar)")
    
    # Muestra una ventana interactiva donde el usuario dibuja la ROI con el ratón
    roi_temp = cv2.selectROI("Selecionar ROI", img, False, False)
    cv2.destroyWindow("Selecionar ROI")
    
    # Si la ROI tiene ancho y alto válidos
    if roi_temp[2] > 0 and roi_temp[3] > 0:
        roi_cords = roi_temp
        x, y, w, h = roi_cords
        
        # Recorta la imagen con las coordenadas seleccionadas
        roi_img = img[y:y+h, x:x+w]
        print(f"ROI: = x={x}, y={y}, w={w}, h={h}")
        return True
    
    return False


def configurate_SIFT(img):
    """
    Crea una interfaz para ajustar los parámetros del detector SIFT.
    Cada slider modifica un parámetro, y la imagen se actualiza en tiempo real.
    Esto NO hace matching, solo sirve para visualizar los keypoints y elegir
    buenos parámetros para SIFT.
    """
    cv2.namedWindow('SIFT configuration')
    
    # Trackbars para ajustar cada parámetro de SIFT
    cv2.createTrackbar('nfeatures', 'SIFT configuration', 500, 2000, nothing)
    cv2.createTrackbar('nOctaveLayers', 'SIFT configuration', 3, 5, nothing)
    cv2.createTrackbar('contrastThr.', 'SIFT configuration', 40, 100, nothing)
    cv2.createTrackbar('edgeThreshold', 'SIFT configuration', 10, 20, nothing)
    cv2.createTrackbar('sigma', 'SIFT configuration', 16, 30, nothing)
    
    print("Ajustar los parametros del SIFT. Presionar q para continuar")
    
    while True:
        # Lectura en tiempo real del valor de cada slider
        params_SIFT['nfeatures'] = nfeat = cv2.getTrackbarPos('nfeatures', 'SIFT configuration')
        params_SIFT['nOctaveLayers'] = nOc = cv2.getTrackbarPos('nOctaveLayers', 'SIFT configuration')
        params_SIFT['contrastThr.'] = conTh = cv2.getTrackbarPos('contrastThr.', 'SIFT configuration') / 1000.0
        params_SIFT['edgeThreshold'] = edge = cv2.getTrackbarPos('edgeThreshold', 'SIFT configuration')
        params_SIFT['sigma'] = sigma = cv2.getTrackbarPos('sigma', 'SIFT configuration') / 10.0
        
        # Crear un detector SIFT con los parámetros ajustados
        sift = cv2.SIFT_create(nfeat, nOc, conTh, edge, sigma)
        
        # Convertir a escala de grises para detectar keypoints
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detectar keypoints y descriptores
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        
        # Dibujar keypoints sobre la imagen
        img_with_keypoints = cv2.drawKeypoints(
            img, 
            keypoints, 
            None, 
            flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
        )
        
        cv2.imshow('SIFT configuration', img_with_keypoints)
        
        # Si el usuario pulsa 'q', termina la configuración
        key = cv2.waitKey(100) & 0xFF
        if key == ord('q'):
            break
    
    cv2.destroyWindow('SIFT configuration')
    print("Parametros de SIFT configurados")


def detect_matches(img_trf, img_roi, num_matches = 10):
    """
    Función principal del ejercicio:
    - Detecta características SIFT en la ROI y en la imagen transformada.
    - Realiza matching con FLANN.
    - Filtra los mejores matches según la regla de Lowe.
    - Calcula la homografía para localizar la ROI en la imagen transformada.
    - Dibuja un rectángulo rojo indicando dónde aparece la ROI.
    - Devuelve la imagen de resultados y la de matches.
    """
    
    # Comprobación de seguridad
    if img_trf is None or img_roi is None:
        print("Error: Falta de ROI o imagen transformada")
        return None
    
    # Crear SIFT con los parámetros ajustados previamente
    sift = cv2.SIFT_create(
        nfeatures=params_SIFT['nfeatures'],
        nOctaveLayers=params_SIFT['nOctaveLayers'],
        contrastThreshold=params_SIFT['contrastThreshold'],
        edgeThreshold=params_SIFT['edgeThreshold'],
        sigma=params_SIFT['sigma']
    )
    
    # Convertir imágenes a escala de grises
    roi_gray = cv2.cvtColor(img_roi, cv2.COLOR_BGR2GRAY)
    trf_gray = cv2.cvtColor(img_trf, cv2.COLOR_BGR2GRAY)
    
    # Detectar keypoints y descriptores en ROI y transformada
    kp_roi, des_roi = sift.detectAndCompute(roi_gray, None)
    kp_trf, des_trf = sift.detectAndCompute(trf_gray, None)
    
    print(f"Roi Keypoints: {len(kp_roi)}, Transformada Keypoints: {len(kp_trf)}")
    
    # Si hay pocos descriptores la homografía no es fiable
    if des_roi is None or des_trf is None or len(kp_roi) < 2 or len(kp_trf) < 2:
        print("Pocas caracteristicas encontradas")
        return None
    
    # FLANN para hacer matching rápido en espacios de alta dimensión
    flann = cv2.FlannBasedMatcher(
        dict(algorithm=1, trees=5), 
        dict(checks=50)
    )
    
    # Encontrar los dos mejores matches para cada descriptor de la ROI
    matches = flann.knnMatch(des_roi, des_trf, k=2)
    
    # Regla de Lowe: solo se considera bueno si el match es claramente mejor que el segundo
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
        
    print(f"Matches buenos: {len(good_matches)}")
    
    # Si no hay suficientes matches, no se puede calcular homografía
    if len(good_matches) < num_matches:
        print(f"Pocos matches ({len(good_matches)} < {num_matches})")
        return None
    
    # Extraer las coordenadas de los puntos coincidentes
    src_pts = np.float32([kp_roi[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_trf[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    # Calcular homografía con RANSAC (permite tolerar outliers)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    if M is None:
        print("No se pudo calcular la homografía")
        return None
    
    # Crear un rectángulo rojo alrededor de la posición estimada de la ROI
    h, w = roi_gray.shape
    pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    
    # Transformar los puntos según la homografía calculada
    dst = cv2.perspectiveTransform(pts, M)
    
    img_result = img_trf.copy()
    img_result = cv2.polylines(img_result, [np.int32(dst)], True, (0, 0, 255), 3)
    
    # Dibuja los matches entre ROI e imagen transformada
    img_matches = cv2.drawMatches(
        img_roi, kp_roi,
        img_trf, kp_trf,
        good_matches[:50],
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    
    print("Roi Encontrado")
    return img_result, img_matches, M, len(good_matches)


In [11]:
# CARGA DE LA IMAGEN ORIGINAL

# Abre una ventana para seleccionar la imagen desde el explorador
ruta = filedialog.askopenfilename(title="Selecciona una imagen")

# Carga la imagen seleccionada en memoria
img_original = cv2.imread(ruta)

# Verificación por si la imagen no se ha podido cargar correctamente
if img_original is None:
    print("Error: no se pudo cargar la imagen")
    exit()

# Reducción de tamaño para facilitar el procesamiento (20% del tamaño original)
img_original = cv2.resize(img_original, None, fx=0.2, fy=0.2)

# Variables globales donde se guardará la ROI seleccionada
roi_cords = None
roi_img = None

# SELECCIÓN DE LA ROI (REGIÓN DE INTERÉS)

# El usuario selecciona manualmente el área que quiere buscar
select_roi(img_original)

# Variable para almacenar la imagen transformada
img_transformada = None

# CREACIÓN DE LA CLASE DE -TRANSFORMACIONES-

transformaciones = Transformaciones(img_original)

transformaciones.distorcion()
img_transformada = transformaciones.distorcioned

#transformaciones.transformacion()
#img_transformada = transformaciones.transformed


# PARÁMETROS INICIALES DEL DETECTOR SIFT

params_SIFT = {
    "nfeatures": 500,          # Número máximo de keypoints a detectar
    "nOctaveLayers": 3,        # Número de capas por octava
    "contrastThreshold": 0.04, # Umbral de contraste para eliminar puntos débiles
    "edgeThreshold": 10,       # Umbral para descartar bordes inestables
    "sigma": 1.6               # Suavizado aplicado en la primera octava
}

# AJUSTE INTERACTIVO DE SIFT POR EL USUARIO

# Se abre una ventana con sliders para modificar los parámetros de SIFT
# Mientras se mueven, se muestran los keypoints actualizados en tiempo real.
configurate_SIFT(img_original)


# DETECCIÓN DE LA ROI EN LA IMAGEN TRANSFORMADA

resultado = detect_matches(img_transformada, roi_img, 5)

# Si no hay resultados suficientes, se informa por pantalla
if resultado is None:
    print("No se encontraron matches suficientes")

else:
    # Si se encuentra la ROI, se obtienen imágenes de resultados:
    # - imgen_result : imagen transformada con cuadro rojo alrededor de la ROI encontrada
    # - imgen_matches : visualización de matches entre ROI y transformada
    # - M : homografía calculada
    # - tam : número de matches buenos
    imgen_result, imgen_matches, M, tam = resultado

    
    # VISUALIZACIÓN FINAL DE RESULTADOS

    while True:
        # Imagen original sin transformar (solo para referencia visual)
        cv2.imshow("Imagen Original", img_original)

        # Imagen mostrando los matches entre ROI y transformada
        cv2.imshow("Imagen Matches", imgen_matches)

        # Imagen transformada con el rectángulo rojo indicando la coincidencia
        cv2.imshow("Imagen Resultados", imgen_result)

        # Espera de teclado — si el usuario pulsa ESC (27), se cierra
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break

    # Cierra todas las ventanas abiertas por OpenCV
    cv2.destroyAllWindows()


Selecionar el Area de Interes (Pulse ENTER para continuar)
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
ROI: = x=373, y=94, w=32, h=45
Ajustar los parametros del SIFT. Presionar q para continuar
Parametros de SIFT configurados
Roi Keypoints: 45, Transformada Keypoints: 1562
Matches buenos: 5
Roi Encontrado


### Significado de los parámetros SIFT
 - nfeatures

Cuántos puntos máximos intenta detectar.

Alto → detecta más detalles (aunque sean débiles).

Bajo → solo los puntos más fuertes.

- contrastThreshold

Controla cuánto contraste debe tener un punto para ser considerado importante.

Bajo → detecta puntos más débiles (más “ruido” pero más matches).

Alto → menos puntos, pero más fiables.

- edgeThreshold

Controla si descarta puntos que solo están en bordes “planos” o alargados.

Bajo → detecta casi todo.

Alto → descarta muchos puntos en bordes simples (muy útil en ropa, logotipos planos, etc.).

- sigma

Suavizado inicial.

Bajo → detecta detalles pequeños.

Alto → detecta estructuras grandes.